# Assignment: Training EEGNet on P300 EEG Data

In this assignment, you will work with real EEG data from a P300 speller experiment and implement the EEGNet architecture to detect P300 responses. The emphasis of this assignment is on understanding and implementing the EEGNet model rather than extensive signal preprocessing.

**Instructions:**
- Complete the provided code scaffolding
- Fill in missing logic where indicated
- Focus especially on the EEGNet architecture and training


## Part 1: Loading and Inspecting the Dataset

In this section, you will load the EEG dataset and inspect its basic structure. The dataset contains continuous EEG recordings along with stimulus and label information.

In [1]:
import scipy.io as sio
import numpy as np
from google.colab import drive
import tensorflow as tf

# Set default data format for Keras backend to 'channels_last'
tf.keras.backend.set_image_data_format('channels_last')

# Load the dataset
# TODO: Update the path if needed
drive.mount('/content/drive')
DATA_PATH = '/content/drive/MyDrive/BCI_Comp_III_Wads_2004/'
data = sio.loadmat(DATA_PATH + 'Subject_A_Train.mat')

# Inspect available keys
print(data.keys())

Mounted at /content/drive
dict_keys(['__header__', '__version__', '__globals__', 'Signal', 'TargetChar', 'Flashing', 'StimulusCode', 'StimulusType'])


## Part 2: Understanding the Experimental Design

The P300 speller paradigm is based on detecting brain responses to rare target stimuli. In this section, you will identify how stimulus timing and labels are encoded in the data.

In [5]:
# TODO: Identify which variables correspond to
# 1. Continuous EEG signal
# 2. Stimulus onset information
# 3. Target vs non-target labels

signal = data['Signal']
flashing = data['Flashing']
stimulus_type = data['StimulusType']

n_epochs, n_samples, n_ch = signal.shape
signal = signal.reshape(n_epochs * n_samples, n_ch).T
flashing = flashing.reshape(-1)
stimulus_type = stimulus_type.reshape(-1) if stimulus_type is not None else None

onsets_id=[]

# Find stimulus onsets (flashing goes from 0 to 1)
for j in range(len(flashing)):
       if j==0:
           if flashing[j]==1:
               onsets_id.append(j)

       else:
           if flashing[j]==1 and flashing[j-1]==0:
               onsets_id.append(j)

labels=[]

if stimulus_type is not None:
    labels.append(stimulus_type[onsets_id])
else:
    labels.append(-1)


labels=np.squeeze(np.array(labels))



# Hint: Look for variables related to stimulus codes and stimulus types


## Part 3: EEG Epoch Extraction

EEGNet does not operate on continuous EEG. Instead, the signal must be segmented into short epochs following each stimulus. This step converts raw EEG into trials suitable for supervised learning.

In [6]:
def extract_epochs(signal, stimulus_onsets, labels, fs=240, t_start=0.0, t_end=0.8):
    """
    Extract EEG epochs around each stimulus onset.

    Parameters:
    - signal: continuous EEG array of shape (time, channels)
    - stimulus_onsets: indices where stimuli occur
    - labels: target/non-target labels per stimulus
    - fs: sampling frequency in Hz
    - t_start: start time (seconds) relative to stimulus
    - t_end: end time (seconds) relative to stimulus

    Returns:
    - epochs: array of shape (num_trials, channels, time)
    - y: corresponding labels
    """
    # TODO: Implement epoch extraction logic
    # Hint: Convert time window to samples using fs

    samples_lower = int(t_start*fs)
    samples_upper = int(t_end*fs)
    samples_per_epoch = samples_upper + samples_lower

    epochs = []


    for onset in stimulus_onsets:
      if (onset + samples_upper) < np.size(signal, axis=1) and onset-samples_lower >=0 :
        epoch = signal[:,onset-samples_lower:onset + samples_upper+1]
        epochs.append(epoch)



    epochs = np.array(epochs)


    print(f"Extracted {len(epochs)} epochs")
    print(f"Epoch shape: {epochs.shape}")

    return {
        'epochs': epochs,
        'labels': labels,
    }

epochs_dict=extract_epochs(signal, onsets_id, labels)

Extracted 15300 epochs
Epoch shape: (15300, 64, 193)


## Part 4: Preparing Data for EEGNet

In this section, you will perform minimal preprocessing to make the data compatible with EEGNet. Extensive signal processing is not required.

In [7]:
def prepare_for_eegnet(epochs):
    """
    Prepare EEG epochs for input into EEGNet.

    Expected input shape: (trials, channels, time)
    Expected output shape: (trials, channels, time, 1) for channels_last
    """
    # TODO: Add singleton dimension required by Conv2D
    # Hint: Use numpy.expand_dims

    # Change axis to -1 for channels_last format
    epochs = np.expand_dims(epochs, axis=-1)
    return epochs

prepared_epochs = prepare_for_eegnet(epochs_dict['epochs'])
X=prepared_epochs
y=epochs_dict['labels']
nb_classes=2

## Part 5: Implementing EEGNet

This is the core part of the assignment. You will implement the EEGNet architecture as discussed in class. Focus on matching the block structure and understanding the role of each layer.

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, DepthwiseConv2D,
                                     SeparableConv2D, BatchNormalization,Activation,
                                     AveragePooling2D, Dropout, Flatten, Dense)
from tensorflow.keras.constraints import max_norm

def EEGNet(nb_classes, Chans, Samples, F1=8, D=2, F2=16, dropoutRate=0.5):
    """
    EEGNet architecture.

    Parameters:
    - nb_classes: number of output classes
    - Chans: number of EEG channels
    - Samples: number of time samples per epoch
    - F1: number of temporal filters
    - D: depth multiplier for spatial filters
    - F2: number of pointwise filters
    """

    # Input is (batch, Chans, Samples, 1) for channels_last
    inputs = Input(shape=(Chans, Samples, 1))

    # Block 1: Temporal Convolution
    # Kernel (1, temporal_filter_length) operates on (height, width) where height=Chans (implicitly across channels) and width=Samples.
    # Using kernel (1, 120) for spatial (channel-wise) and temporal (time-wise) dimensions respectively.
    block1 = Conv2D(F1, (1, 120), padding = 'same', use_bias = False, data_format='channels_last')(inputs)
    block1 = BatchNormalization(axis=-1)(block1) # axis=-1 is the channel axis for channels_last

    # Block 1: Spatial Convolution
    # Kernel (Chans, 1) operates across channels (height) dimension.
    block1 = DepthwiseConv2D((Chans, 1), use_bias = False, depth_multiplier = D, depthwise_constraint = max_norm(1.), data_format='channels_last')(block1)
    block1 = BatchNormalization(axis=-1)(block1)
    block1 = Activation('elu')(block1)
    block1 = AveragePooling2D((1, 4), data_format='channels_last')(block1)
    block1 = Dropout(dropoutRate)(block1)

    # Block 2: Separable Convolution
    # Separable Conv for temporal features (similar to first Conv2D)
    block2 = SeparableConv2D(F2, (1, 16), use_bias = False, padding = 'same', data_format='channels_last')(block1)
    block2 = BatchNormalization(axis=-1)(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((1, 8), data_format='channels_last')(block2)
    block2 = Dropout(dropoutRate)(block2)

    # Classification
    flatten = Flatten(name = 'flatten')(block2)
    dense = Dense(nb_classes, name = 'dense')(flatten)
    softmax = Activation('softmax', name = 'softmax')(dense)

    return Model(inputs=inputs, outputs=softmax)


# TODO: Instantiate the EEGNet model and print the summary

model = EEGNet(
    nb_classes=2,
    Chans=64,
    Samples=193
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 64, 193, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 64, 193, 8)     │           960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 193, 8)     │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d                │ (None, 1, 193, 16)     │         1,024 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1, 193, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 1, 193, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 1, 48, 16)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 48, 16)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d                │ (None, 1, 48, 16)      │           512 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1, 48, 16)      │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 1, 48, 16)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 1, 6, 16)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1, 6, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │           194 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax (Activation)            │ (None, 2)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,850 (11.13 KB)

 Trainable params: 2,770 (10.82 KB)

 Non-trainable params: 80 (320.00 B)

## Part 6: Training the Model

In this section, you will train EEGNet to distinguish between P300 and non-P300 EEG epochs.

In [13]:
# TODO: Split the dataset into training and validation sets
# TODO: Compile the model with an appropriate loss and optimizer
# Hint: Use categorical cross-entropy and Adam optimizer
# TODO: Train the model and store the training history

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam


# Convert labels to one-hot (required for categorical_crossentropy)
y_cat = to_categorical(y, num_classes=nb_classes)

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y_cat,
    test_size=0.2,
    random_state=42,
    stratify=y_cat
)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=16,
    validation_data=(X_val, y_val),
    verbose=1
)



Epoch 1/20
765/765 ━━━━━━━━━━━━━━━━━━━━ 378s 489ms/step - accuracy: 0.8320 - loss: 0.4303 - val_accuracy: 0.8304 - val_loss: 0.4183
Epoch 2/20
765/765 ━━━━━━━━━━━━━━━━━━━━ 372s 487ms/step - accuracy: 0.8376 - loss: 0.4059 - val_accuracy: 0.8320 - val_loss: 0.3988
Epoch 3/20
765/765 ━━━━━━━━━━━━━━━━━━━━ 366s 478ms/step - accuracy: 0.8329 - loss: 0.4108 - val_accuracy: 0.8324 - val_loss: 0.4090
Epoch 4/20
765/765 ━━━━━━━━━━━━━━━━━━━━ 363s 474ms/step - accuracy: 0.8282 - loss: 0.4146 - val_accuracy: 0.8359 - val_loss: 0.3943
Epoch 5/20
765/765 ━━━━━━━━━━━━━━━━━━━━ 389s 483ms/step - accuracy: 0.8358 - loss: 0.4018 - val_accuracy: 0.8386 - val_loss: 0.3909
Epoch 6/20
765/765 ━━━━━━━━━━━━━━━━━━━━ 361s 471ms/step - accuracy: 0.8448 - loss: 0.3849 - val_accuracy: 0.8402 - val_loss: 0.3896
Epoch 7/20
765/765 ━━━━━━━━━━━━━━━━━━━━ 384s 474ms/step - accuracy: 0.8412 - loss: 0.3845 - val_accuracy: 0.8363 - val_loss: 0.3820
Epoch 8/20
765/765 ━━━━━━━━━━━━━━━━━━━━ 366s 478ms/step - accuracy: 0.8459 -